In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [10]:
# Create the Cohere chat model
from langchain_cohere.chat_models import ChatCohere
os.environ["COHERE_API_KEY"] = COHERE_API_KEY
chat = ChatCohere(model="command-r-plus", temperature=0.3)

Tool 1: web search

In [25]:
from langchain_community.tools import TavilySearchResults

os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

internet_search = TavilySearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."


# from langchain_core.pydantic_v1 import BaseModel, Field
# class TavilySearchInput(BaseModel):
#     query: str = Field(description="Query to search the internet with")
# internet_search.args_schema = TavilySearchInput

In [23]:
from langchain_community.tools import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)

In [24]:
tool.invoke({'query': 'who won the last french open'})

[{'url': 'https://www.sportskeeda.com/tennis/who-last-french-players-win-french-open',
  'content': 'Francoise Durr, the legendary French player, was the last Frenchwoman to win the French Open in 1967. The duo of Mary Peirce and Martina Hingis also won the doubles title that same year. Expand Tweet'},
 {'url': "https://en.wikipedia.org/wiki/List_of_French_Open_Men's_Singles_champions",
  'content': "Ivan Lendl won his first title in 1984, before losing the following year to Wilander in the final and won two consecutive titles in 1986 and 1987.[19] Michael Chang became the youngest man to win the French Open when he beat Stefan Edberg in 1989.[21]\nAmerican Jim Courier won consecutive titles in 1991 and 1992 before Spaniard Sergi Bruguera repeated the feat in 1993 and 1994.[22][23] Gustavo Kuerten won three titles in 1997, 2000 and 2001.[19] 2005 marked Rafael Nadal's first French Open; he won four consecutive titles from 2005 to 2008.[24] These results are not recognised by the FFT or

Tool 2: plot

In [26]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
repl_tool.name = "python_interpreter"

# from langchain_core.pydantic_v1 import BaseModel, Field
class ToolInput(BaseModel):
    code: str = Field(description="Python code to execute.")
repl_tool.args_schema = ToolInput

Agent

In [27]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate

# Create the prompt
prompt = ChatPromptTemplate.from_template("{input}")


# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=chat,
    tools=[internet_search, repl_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, tools=[internet_search, repl_tool], verbose=True)

In [28]:
try:
    result = agent_executor.invoke({
        "input": "Create a plot of the monthly average of gold price in swiss franc by gramm for the last 10 years",
    })
except AttributeError as e:
    print(f"Error: {e}")



> Entering new AgentExecutor chain...
Error: type object 'ToolInput' has no attribute 'model_json_schema'
